# Computational Drug Discovery Tutorial

Welcome to this hands-on notebook accompanying the blog tutorial.  
We'll cover:

1. **Environment Setup**  
2. **Protein Preparation**  
3. **Ligand Preparation**  
4. **Docking with AutoDock Vina**  
5. **Optional Mini Virtual Screening**  
6. **Basic ML Model**  
7. **Next Steps & Wrap-Up**  

---

## 1. Environment & Initial Setup



In [1]:
# If running on Colab, install necessary packages here:
# (Uncomment the lines below if you need them)

# !pip install rdkit-pypi openbabel deepchem pyimolecule
# !apt-get update
# !apt-get install -y openbabel

import sys
import os

# Check Python and library versions
print("Python version:", sys.version)

try:
    import rdkit
    import deepchem
    from rdkit import Chem
    print("RDKit version:", rdkit.__version__)
    print("DeepChem version:", deepchem.__version__)
except ImportError as e:
    print("Some packages are missing. Please install them before proceeding.")
    raise e

# Define a data directory (adjust to your structure)
DATA_DIR = "./data"
RESULTS_DIR = "./results"

os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)

print("Data directory:", DATA_DIR)
print("Results directory:", RESULTS_DIR)


Python version: 3.11.11 | packaged by conda-forge | (main, Dec  5 2024, 14:17:24) [GCC 13.3.0]



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/joe/miniconda3/envs/deepchem-env/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/joe/miniconda3/envs/deepchem-env/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/joe/miniconda3/envs/deepchem-env/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in st

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/joe/miniconda3/envs/deepchem-env/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/joe/miniconda3/envs/deepchem-env/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/joe/miniconda3/envs/deepchem-env/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in st

AttributeError: _ARRAY_API not found

...

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/joe/miniconda3/envs/deepchem-env/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/joe/miniconda3/envs/deepchem-env/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/joe/miniconda3/envs/deepchem-env/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in st

AttributeError: _ARRAY_API not found

Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading some PyTorch models, missing a dependency. No module named 'torch'
No module named 'torch'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch'
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'torch'
Skipped loading some Jax models, missing a dependency. No module named 'jax'
Skipped loading some PyTorch models, missing a dependency. No module named 'tensorflow'


RDKit version: 2022.09.5
DeepChem version: 2.8.0
Data directory: ./data
Results directory: ./results
